<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_10_23_Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DataFrame

In [1]:
# Install Spark 3.2.0
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!rm -f *.tgz*

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

!pip install -q findspark
!pip install -q pyspark==3.2.0

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 13.5 MB/s 


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

from datetime import datetime, date
import pandas as pd

spark=SparkSession.builder.appName("local[*]").getOrCreate()

##Create a PySpark DataFrame from a list of rows.

In [3]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

print(df)

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]


##Create a PySpark DataFrame with an explicit schema.

In [4]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')

print(df)

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]


##Create a PySpark DataFrame from a pandas DataFrame

In [5]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
print(pandas_df)

df = spark.createDataFrame(pandas_df)
print(df)

   a    b        c           d                   e
0  1  2.0  string1  2000-01-01 2000-01-01 12:00:00
1  2  3.0  string2  2000-02-01 2000-01-02 12:00:00
2  3  4.0  string3  2000-03-01 2000-01-03 12:00:00
DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]


## Read data with Pandas

### CSV file

In [6]:
pandas_df = pd.read_csv('sample_data/california_housing_test.csv')

california_housing_df = spark.createDataFrame(pandas_df)

california_housing_df.show()
california_housing_df.printSchema()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
|  -118.36|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|       6.1359|          330000.0|
|  -119.67|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0|
|  -119.56|   36.51|    

###JSON file

In [7]:
pandas_df = pd.read_json('sample_data/anscombe.json')

anscombe_df = spark.createDataFrame(pandas_df)
anscombe_df.show()
anscombe_df.printSchema()

+------+---+------------------+
|Series|  X|                 Y|
+------+---+------------------+
|     I| 10|              8.04|
|     I|  8|              6.95|
|     I| 13|              7.58|
|     I|  9|              8.81|
|     I| 11|              8.33|
|     I| 14|              9.96|
|     I|  6|              7.24|
|     I|  4|              4.26|
|     I| 12|             10.84|
|     I|  7|4.8100000000000005|
|     I|  5|              5.68|
|    II| 10|              9.14|
|    II|  8|              8.14|
|    II| 13|              8.74|
|    II|  9|              8.77|
|    II| 11|              9.26|
|    II| 14|               8.1|
|    II|  6|              6.13|
|    II|  4|               3.1|
|    II| 12|              9.13|
+------+---+------------------+
only showing top 20 rows

root
 |-- Series: string (nullable = true)
 |-- X: long (nullable = true)
 |-- Y: double (nullable = true)



In [8]:
anscombe_df.show(2, vertical=True)

-RECORD 0------
 Series | I    
 X      | 10   
 Y      | 8.04 
-RECORD 1------
 Series | I    
 X      | 8    
 Y      | 6.95 
only showing top 2 rows



In [9]:
anscombe_df.columns

['Series', 'X', 'Y']

In [10]:
anscombe_df.take(3)

[Row(Series='I', X=10, Y=8.04),
 Row(Series='I', X=8, Y=6.95),
 Row(Series='I', X=13, Y=7.58)]

In [11]:
california_housing_df.take(3)

[Row(longitude=-122.05, latitude=37.37, housing_median_age=27.0, total_rooms=3885.0, total_bedrooms=661.0, population=1537.0, households=606.0, median_income=6.6085, median_house_value=344700.0),
 Row(longitude=-118.3, latitude=34.26, housing_median_age=43.0, total_rooms=1510.0, total_bedrooms=310.0, population=809.0, households=277.0, median_income=3.599, median_house_value=176500.0),
 Row(longitude=-117.81, latitude=33.78, housing_median_age=27.0, total_rooms=3589.0, total_bedrooms=507.0, population=1484.0, households=495.0, median_income=5.7934, median_house_value=270500.0)]

##Selecting and Accessing Data

PySpark DataFrame viene valutato in "modo minimale" e la semplice selezione di una colonna non attiva un calcolo ma restituisce un'istanza di colonna.


In [12]:
df.a

Column<'a'>

La maggior parte delle operazioni per colonna restituisce colonne.

In [13]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

type(df.c) == type(upper(df.c)) == type(df.c.isNull())

True

Queste colonne possono essere utilizzate per selezionare le colonne da un DataFrame. Per esempio, **DataFrame.select()** accetta le istanze di *Column* e restituisce un altro DataFrame.

In [14]:
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



Assegna una nuova istanza di colonna.

In [15]:
df.withColumn('upperC', upper(df.c)).show()

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e| upperC|
+---+---+-------+----------+-------------------+-------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|STRING1|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|STRING2|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|STRING3|
+---+---+-------+----------+-------------------+-------+



Per selezionare un sottoinsieme di righe, si utilizza **DataFrame.filter()**.

In [16]:
df.filter(df.a == 1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



##Applying a Function

PySpark supporta varie UDF (*user-defined function*) e API per consentire agli utenti di eseguire funzioni native di Python.L'esempio seguente consente agli utenti di utilizzare direttamente le API in una serie di panda all'interno della funzione nativa di Python.

In [17]:
import pandas
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

df.select(pandas_plus_one(df.a)).show()

+------------------+
|pandas_plus_one(a)|
+------------------+
|                 2|
|                 3|
|                 4|
+------------------+



Un altro esempio è **DataFrame.mapInPandas** che consente agli utenti di utilizzare direttamente le API in un DataFrame panda senza alcuna restrizione come la lunghezza del risultato.

In [18]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a == 1]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



##Grouping Data

PySpark DataFrame fornisce anche un modo per gestire i dati raggruppati utilizzando l'approccio comune, la strategia *split-apply-combine*. Raggruppa i dati in base a una determinata condizione applica una funzione a ciascun gruppo e quindi li combina di nuovo al DataFrame.

In [52]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], 
    schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



Raggruppamento e quindi applicazione della funzione **avg()** ai gruppi risultanti.

In [20]:
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [21]:
df.groupby('fruit').avg().show()

+------+------------------+------------------+
| fruit|           avg(v1)|           avg(v2)|
+------+------------------+------------------+
| grape|               6.0|              60.0|
|banana|3.3333333333333335|33.333333333333336|
|carrot| 4.666666666666667|46.666666666666664|
+------+------------------+------------------+



Si può applicare anche una funzione nativa Python a ciascun gruppo utilizzando l'API pandas.

In [22]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())

df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  0| 60|
| blue|banana| -1| 20|
| blue| grape|  1| 40|
|  red|banana| -3| 10|
|  red|carrot| -1| 30|
|  red|carrot|  0| 50|
|  red|banana|  2| 70|
|  red| grape|  3| 80|
+-----+------+---+---+



Co-raggruppamento e applicazione di una funzione.

---
`pandas.merge_asof(left, right, on=None, left_on=None, right_on=None, left_index=False, right_index=False, by=None, left_by=None, right_by=None, suffixes=('_x', '_y'), tolerance=None, allow_exact_matches=True, direction='backward')`

---
L'unione asof è simile a un left-join tranne per il fatto che abbiniamo la chiave più vicina anziché le chiavi uguali.


In [23]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0), (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ('time', 'id', 'v1'))
df1.show()

df2 = spark.createDataFrame(
    [(20000101, 1, 'x'), (20000101, 2, 'y')],
    ('time', 'id', 'v2'))
df2.show()

def asof_join(l, r):
    return pd.merge_asof(l, r, on='time', by='id')

df1.groupby('id').cogroup(df2.groupby('id')).applyInPandas(
    asof_join, schema='time int, id int, v1 double, v2 string').show()

+--------+---+---+
|    time| id| v1|
+--------+---+---+
|20000101|  1|1.0|
|20000101|  2|2.0|
|20000102|  1|3.0|
|20000102|  2|4.0|
+--------+---+---+

+--------+---+---+
|    time| id| v2|
+--------+---+---+
|20000101|  1|  x|
|20000101|  2|  y|
+--------+---+---+

+--------+---+---+---+
|    time| id| v1| v2|
+--------+---+---+---+
|20000101|  1|1.0|  x|
|20000102|  1|3.0|  x|
|20000101|  2|2.0|  y|
|20000102|  2|4.0|  y|
+--------+---+---+---+



In [53]:
pandas_df = df.to_pandas_on_spark()
pandas_df

,color,fruit,v1,v2
0,red,banana,1,10
1,blue,banana,2,20
2,red,carrot,3,30
3,blue,grape,4,40
4,red,carrot,5,50
5,black,carrot,6,60
6,red,banana,7,70
7,red,grape,8,80


##Getting Data in/out

In [54]:
!rm -rf foo*

df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], 
    schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



###Read & Write **CSV**

In [35]:
df.write.csv('foo.csv')
spark.read.csv('foo.csv').show()

+-----+------+---+---+
|  _c0|   _c1|_c2|_c3|
+-----+------+---+---+
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
+-----+------+---+---+



###Write & Read **Parquet**

In [26]:
df.write.parquet('foo.parquet')
spark.read.parquet('foo.parquet').show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
+-----+------+---+---+



###Read & Write **ORC**

In [27]:
df.write.orc('foo.orc')
spark.read.orc('foo.orc').show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
+-----+------+---+---+



In [41]:
!ls -la foo*

-rw-r--r-- 1 root root 165 Oct 22 17:39 foo_pandas.csv


##Working with SQL

DataFrame e Spark SQL condividono lo stesso motore di esecuzione in modo che possano essere utilizzati in modo intercambiabile senza problemi. Ad esempio, si può registrare un DataFrame come tabella ed eseguire facilmente una query SQL:

In [29]:
df.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



Inoltre, le UDF possono essere registrate e richiamate in SQL immediatamente:

In [30]:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(v1) FROM tableA").show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+



Queste espressioni SQL possono essere mescolate direttamente e utilizzate come colonne PySpark.

In [31]:
from pyspark.sql.functions import expr

df.selectExpr('add_one(v1)').show()
df.select(expr('count(*)') > 0).show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+

+--------------+
|(count(1) > 0)|
+--------------+
|          true|
+--------------+

